# Task 1

In [27]:
import sys
sys.path.append('../deliverable')
import run_model as rm
import matplotlib.pyplot as plt
import numpy as np
import joblib
from sklearn.metrics import mean_squared_error

In [28]:
# load the data
X,y = rm.load_data("../data/data.npz")

# Create X matrix
ones = np.ones((X.shape[0], 1))
X_ = np.column_stack((ones,X,np.cos(X[:,1]),X[:,0]**2))

# Calculate coefficients directly
theta = np.linalg.inv(X_.T.dot(X_)).dot(X_.T).dot(y)

# predict using coefficients
y_pred = X_.dot(theta)

MSE = rm.evaluate_predictions(y, y_pred)

print("theta_0 = {}, theta_1 = {}, theta_2 = {}, theta_3 = {}".format(theta[0], theta[1], theta[2], theta[3]))
print("MSE = {}".format(MSE))

theta_0 = 1.7006171582309046, theta_1 = -0.14325050307474513, theta_2 = -0.623183897160942, theta_3 = 0.018956308893969535
MSE = 1.2418258937189481


In [38]:
#Save the model
joblib.dump((X_,theta),"../deliverable/linear_regression_model.pickle")

['../deliverable/linear_regression_model.pickle']

# Task 2

##### Split the data into test and train sets

In [30]:
from sklearn.model_selection import train_test_split

# Split the data into test and train sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

##### We are going to use MLP Regressor for the second task. To find the best hyperparameters GridSerachCV is going to be used

In [32]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

param_list = {"hidden_layer_sizes": [(1,),(50,)], "activation": ["tanh", "relu"], "solver": ["lbfgs"], "alpha": [0.00005,0.0005]}
gridCV = GridSearchCV(estimator=MLPRegressor(max_iter=90000), param_grid=param_list)

gridCV.fit(X_train, y_train)

GridSearchCV(estimator=MLPRegressor(max_iter=90000),
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [5e-05, 0.0005],
                         'hidden_layer_sizes': [(1,), (50,)],
                         'solver': ['lbfgs']})

In [36]:
print("The best parameters found are {} ".format(gridCV.best_params_))

# Save the model
joblib_file = "../deliverable/nonlinear_model.pickle"
joblib.dump(gridCV,joblib_file)

The best parameters found are {'activation': 'tanh', 'alpha': 5e-05, 'hidden_layer_sizes': (50,), 'solver': 'lbfgs'} 


['../deliverable/nonlinear_model.pickle']

# Task 3

##### Comparing the performance of the two models with the test set

In [34]:
baseline_model = rm.load_model("../deliverable/baseline_model.pickle")

y_pred_baseline_model = baseline_model.predict(X_test)
y_pred_MLP_model = gridCV.predict(X_test)

MSE_baseline_model = rm.evaluate_predictions(y_test, y_pred_baseline_model)
MSE_MLP_model = rm.evaluate_predictions(y_test, y_pred_MLP_model)

print("The MSE of the Baseline Model on the test set is MSE = {} ".format(MSE_baseline_model))
print("The MSE of the MLP Model on the test set is MSE = {} ".format(MSE_MLP_model))

The MSE of the Baseline Model on the test set is MSE = 0.014302971711671629 
The MSE of the MLP Model on the test set is MSE = 0.015776781583212104 
